In [124]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

## 데이터 불러오기

In [154]:
mydata = pd.read_csv("data_main_weather.csv")

In [112]:
pd.set_option('mode.chained_assignment',  None)

In [155]:
# 이상치 처리1
filtering_outlier = mydata.query('revenue > 180000000')
filtering_outlier_index = list(filtering_outlier.index)

for i in filtering_outlier_index:
    tmp = mydata.loc[i,'mother_code']
    mydata.loc[i,'revenue'] = mydata[mydata['mother_code'] == tmp].sort_values(by='revenue' ,ascending=False)['revenue'].iloc[2]

In [ ]:
# 이상치 처리2

"""
filtering_outlier = mydata.query('revenue > 180000000')
filtering_outlier_index = list(filtering_outlier.index)

for i in filtering_outlier_index:
    mydata.loc[i,'revenue'] = mydata.loc[i-1,'revenue']
"""

In [225]:
mydata[mydata['revenue'] > 180000000]

,Unnamed: 0,datetime,year,month,date,dayofweek,time,hour,minute,holiday_1,...,revenue,sex,installment,code,repeat,holiday,m_temp,l_temp,h_temp,precipitation
35784,35784,2019-12-07 01:40:00,2019,12,7,5,01:40:00,1,40,1,...,187419000.0,무관,무관,201322,2,2019-12-07,0.1,-6.5,6.1,0.0
36267,36267,2019-12-11 00:00:00,2019,12,11,2,00:00:00,0,0,0,...,187419000.0,무관,무관,201322,3,2019-12-11,6.9,1.8,13.3,0.2
37356,37356,2019-12-21 17:40:00,2019,12,21,5,17:40:00,17,40,1,...,187419000.0,무관,무관,201322,2,2019-12-21,0.2,-4.1,4.7,0.1


## 이상치 제거 후 랜덤포레스트 성능 평가(MAPE)

In [226]:
mydata1 = mydata.dropna()
mydata1 = mydata1.drop(['holiday','Unnamed: 0', 'datetime','name', 'code','time'], axis=1)
mydata1 = pd.get_dummies(mydata1)

In [230]:
mydata1

,year,month,date,dayofweek,hour,minute,holiday_1,exposure,mother_code,price,...,category_이미용,category_잡화,category_주방,category_침구,sex_남성,sex_무관,sex_여성,installment_무관,installment_무이자,installment_일시불
0,2019,1,1,1,6,0,2,20.0,100346,39900.0,...,0,0,0,0,1,0,0,1,0,0
1,2019,1,1,1,6,0,2,0.0,100346,39900.0,...,0,0,0,0,0,0,1,1,0,0
2,2019,1,1,1,6,20,2,20.0,100346,39900.0,...,0,0,0,0,1,0,0,1,0,0
3,2019,1,1,1,6,20,2,0.0,100346,39900.0,...,0,0,0,0,0,0,1,1,0,0
4,2019,1,1,1,6,40,2,20.0,100346,39900.0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38299,2019,12,31,1,23,40,0,0.0,100448,148000.0,...,0,0,1,0,0,1,0,1,0,0
38300,2020,1,1,2,0,0,2,20.0,100448,178000.0,...,0,0,1,0,0,1,0,1,0,0
38301,2020,1,1,2,0,0,2,0.0,100448,168000.0,...,0,0,1,0,0,1,0,1,0,0
38302,2020,1,1,2,0,0,2,0.0,100448,158000.0,...,0,0,1,0,0,1,0,1,0,0


In [231]:
list(mydata1.columns.values)

['year',
 'month',
 'date',
 'dayofweek',
 'hour',
 'minute',
 'holiday_1',
 'exposure',
 'mother_code',
 'price',
 'count',
 'revenue',
 'repeat',
 'm_temp',
 'l_temp',
 'h_temp',
 'precipitation',
 'category_가구',
 'category_가전',
 'category_건강기능',
 'category_농수축',
 'category_생활용품',
 'category_속옷',
 'category_의류',
 'category_이미용',
 'category_잡화',
 'category_주방',
 'category_침구',
 'sex_남성',
 'sex_무관',
 'sex_여성',
 'installment_무관',
 'installment_무이자',
 'installment_일시불']

In [232]:
col_list = ['year',
 'month',
 'date',
 'dayofweek',
 'hour',
 'minute',
 'holiday_1',
 'exposure',
 'mother_code',
 'price',
 'count',
 'repeat',
 'm_temp',
 'l_temp',
 'h_temp',
 'precipitation',
 'category_가구',
 'category_가전',
 'category_건강기능',
 'category_농수축',
 'category_생활용품',
 'category_속옷',
 'category_의류',
 'category_이미용',
 'category_잡화',
 'category_주방',
 'category_침구',
 'sex_남성',
 'sex_무관',
 'sex_여성',
 'installment_무관',
 'installment_무이자',
 'installment_일시불',
 'revenue']

In [233]:
mydata1 = mydata1.reindex(columns = col_list)

In [235]:
"""
# 트레이닝 테스트 셋
X = mydata1.iloc[:,:-1]
y = mydata1.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=156)
"""

In [255]:
mydata1[mydata1['month']!=6]

,year,month,date,dayofweek,hour,minute,holiday_1,exposure,mother_code,price,...,category_잡화,category_주방,category_침구,sex_남성,sex_무관,sex_여성,installment_무관,installment_무이자,installment_일시불,revenue
0,2019,1,1,1,6,0,2,20.0,100346,39900.0,...,0,0,0,1,0,0,1,0,0,2099000.0
1,2019,1,1,1,6,0,2,0.0,100346,39900.0,...,0,0,0,0,0,1,1,0,0,4371000.0
2,2019,1,1,1,6,20,2,20.0,100346,39900.0,...,0,0,0,1,0,0,1,0,0,3262000.0
3,2019,1,1,1,6,20,2,0.0,100346,39900.0,...,0,0,0,0,0,1,1,0,0,6955000.0
4,2019,1,1,1,6,40,2,20.0,100346,39900.0,...,0,0,0,1,0,0,1,0,0,6672000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38299,2019,12,31,1,23,40,0,0.0,100448,148000.0,...,0,1,0,0,1,0,1,0,0,10157000.0
38300,2020,1,1,2,0,0,2,20.0,100448,178000.0,...,0,1,0,0,1,0,1,0,0,50929000.0
38301,2020,1,1,2,0,0,2,0.0,100448,168000.0,...,0,1,0,0,1,0,1,0,0,104392000.0
38302,2020,1,1,2,0,0,2,0.0,100448,158000.0,...,0,1,0,0,1,0,1,0,0,13765000.0


In [258]:
X_test = mydata1[mydata1['month']==6].iloc[:,:-1]
X_train = mydata1[mydata1['month']!=6].iloc[:,:-1]
y_test = mydata1[mydata1['month']==6].iloc[:,-1]
y_train = mydata1[mydata1['month']!=6].iloc[:,-1]

In [259]:
y_train

0          2099000.0
1          4371000.0
2          3262000.0
3          6955000.0
4          6672000.0
            ...     
38299     10157000.0
38300     50929000.0
38301    104392000.0
38302     13765000.0
38303     46608000.0
Name: revenue, Length: 34361, dtype: float64

In [261]:
X_test

,year,month,date,dayofweek,hour,minute,holiday_1,exposure,mother_code,price,...,category_이미용,category_잡화,category_주방,category_침구,sex_남성,sex_무관,sex_여성,installment_무관,installment_무이자,installment_일시불
16109,2019,6,1,5,1,0,1,20.0,100380,39900.0,...,0,0,0,0,1,0,0,1,0,0
16110,2019,6,1,5,1,0,1,0.0,100380,39900.0,...,0,0,0,0,0,0,1,1,0,0
16111,2019,6,1,5,1,20,1,20.0,100380,39900.0,...,0,0,0,0,1,0,0,1,0,0
16112,2019,6,1,5,1,20,1,0.0,100380,39900.0,...,0,0,0,0,0,0,1,1,0,0
16113,2019,6,1,5,1,40,1,13.0,100380,39900.0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19177,2019,6,30,6,22,20,1,20.0,100098,119000.0,...,0,0,0,0,0,1,0,1,0,0
19178,2019,6,30,6,22,40,1,20.0,100098,119000.0,...,0,0,0,0,0,1,0,1,0,0
19179,2019,6,30,6,23,0,1,20.0,100098,119000.0,...,0,0,0,0,0,1,0,1,0,0
19180,2019,6,30,6,23,20,1,20.0,100664,79900.0,...,0,0,0,0,0,1,0,1,0,0


In [239]:
rf = RandomForestRegressor(random_state=0, n_estimators=100)

In [263]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [264]:
y_pred = rf.predict(X_test)

In [265]:
y_pred[0:30]

array([ 4491500.,  2980610., 10126870.,  6086000.,  7015770.,  5625630.,
       12066860., 15713620., 21918120., 11705480., 28063000., 37178900.,
       52849910., 79276200., 97266500., 19710180., 19628830., 55707330.,
       68021080., 17672570., 14777610., 28949600., 24601150., 30086600.,
       24957310.,  7029330., 26893590., 48274890., 19640470., 28870710.])

In [266]:
np.array(y_test)[0:30]

array([ 4497000.,  2968000., 10164000.,  6025000.,  7010000.,  5620000.,
       11954000., 15705000., 21884000., 11685000., 27955000., 37021000.,
       52972000., 79733000., 96380000., 19602000., 19503000., 55647000.,
       71581000., 17823000., 14832000., 28914000., 24728000., 30201000.,
       25004000.,  7078000., 27023000., 48719000., 19735000., 28965000.])

In [171]:
# MAPE 평가 지표
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [173]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [267]:
mean_absolute_percentage_error(y_test, y_pred)

0.9169447200972656

------------------

## 이상치 제거 후 랜덤포레스트 성능 평가(RMSE)

In [248]:
y_target = mydata1['revenue']
y_target.fillna(0, inplace=True)
X_data = mydata1.drop(['revenue'], axis=1)
#X_data = data.drop(['revenue', 'Unnamed: 0', 'datetime','name', 'code','time','h_temp','l_temp'], axis=1, inplace=False)
#X_data = data.drop(['revenue', 'Unnamed: 0', 'datetime','name', 'code','time','h_temp','l_temp','precipitation'], axis=1, inplace=False)
X_data.fillna(0, inplace=True)

In [249]:
X_data = pd.get_dummies(X_data)

In [118]:
rf = RandomForestRegressor(random_state=0, n_estimators=100)

In [268]:
neg_mse_scores = cross_val_score(rf, X_data, y_target, scoring="neg_mean_squared_error", cv=20)
rmse_scores = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

In [269]:
print('10교차 검증의 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
print('10교차 검증의 개별 RMSE scores: ', np.round(rmse_scores, 2))
print('10교차 검증의 평균 RMSE scores: {0:.3f}'.format(avg_rmse))

10교차 검증의 개별 Negative MSE scores:  [-6.15826570e+11 -5.33582162e+11 -3.74706586e+11 -1.91714765e+11
 -1.29095879e+11 -1.26047486e+11 -2.04117943e+11 -1.94830420e+11
 -6.91435780e+11 -2.15897754e+11 -1.04768917e+12 -2.99399878e+11
 -2.71309556e+11 -1.02290205e+12 -6.76567325e+11 -7.36913620e+11
 -1.60114288e+12 -7.71381765e+12 -6.09212600e+12 -8.04163293e+12]
10교차 검증의 개별 RMSE scores:  [ 784746.18  730467.08  612132.82  437852.45  359299.15  355031.67
  451794.14  441395.99  831526.18  464647.99 1023566.89  547174.45
  520873.84 1011386.2   822537.13  858436.73 1265362.75 2777376.04
 2468223.25 2835777.31]
10교차 검증의 평균 RMSE scores: 979980.411
